### Process and save CLM daily averages
This script takes hourly CLM outputs as PFB files and computes the daily averages to be saved as PFB files.

Inputs:
- Directory where CLM outputs are and directory where you want to save output
- Hourly PFB files of CLM outputs
- Water year and day start/end

Outputs:
- PFB files for daily average of each variable:  
    - Latent heat (LH) – CLM out layer 1 [W/m^2]
    - Sensible heat flux (SH) – CLM out layer 3 [W/m^2]
    - ground evaporation without condensation (qflx_grnd) – CLM out layer 6 [mm/s]
    - Vegetation transpiration (qflx_trans) – CLM out layer 9 [mm/s]
    - Snow water equivalent (SWE) – CLM out layer 11 [mm]
    - Ground temperature (Tgrnd) – CLM out layer 12 [K] skin temp
    - Soil temperature (Tsoil) – CLM out layer 14 - last layer [K] *** This script outputs all soil temp layers as a 3D array
    - Evapotranspiration from qflx_evap_tot - CLM out layer 5 [mm/s] *** NOT converted in this script  


Notes (10/23/22):  
- Figure out which vars are sum (accumulated) and which are averages
    - Updated this 10/25, but should get this checked
- ET from qflx_evap_tot is just the direct CLM output. Do we want to convert this here? Is there a different way we want to calculate ET? (PFTools' `calculate_evapotranspiration` currently has weird units, so was recommended against using this).  
  - *** Also, technically we can just output the qflx_evap_tot just the same as we would the other CLM variables and delete the ET part...
- Soil Temp in this script is saved as a 3D array (temp for each soil layer), which is different than the Fortran scripts which only saved the CLM layer 14, temp @5cm


In [2]:
import numpy as np
from parflow import Run
import sys
from parflow.tools.io import read_pfb,write_pfb
import parflow.tools.hydrology as hydro

In [3]:
NCLMOUTPUTS = 13 + 4 #13 (number variables) + number of layers over which CLM is active, NZ root

#these 3 entries (year, day start and day end) will eventually be argv to the script so that it can be run from bash script
water_year = 1999
day_start = 364 #day_start = 0 is the first day of the water year, Oct 1 (e.g., day_start = 2 starts at hour 49)
day_end = 365 #day_end = 365 is the final day of the water year, Sept 30

# water_year = int(sys.argv[1])
# day_start = int(sys.argv[1])
# day_end = int(sys.argv[1])

# path to CLM outputs outputs
#path_outputs = '/glade/scratch/tijerina/CONUS2/spinup_WY2003/run_inputs/' #f'/WY{water_year}/'
path_outputs = f'/hydrodata/PFCLM/Taylor/simulations/{water_year}/' #f'/WY{water_year}/'


# PFCLM run name
runname = f'Taylor_{water_year}' #f'CONUS2_{water_year}'

# directory to save averages to
#directory_out = f'/glade/scratch/tijerina/CONUS2/spinup_WY2003/averages'
directory_out = '/home/dtt2/CONUS2/analysis_scripts/Taylor_test_outputs'

# # path to CLM outputs and pfidb
# path_outputs = '/glade/scratch/tijerina/CONUS2/spinup_WY2003/run_inputs/'#f'/WY{water_year}/'
# runname = 'spinup.wy2003' #f'CONUS2_{water_year}'

# # directory to save averages to
# directory_out = f'/glade/scratch/tijerina/CONUS2/spinup_WY2003/averages'

In [3]:
### Keeping this here for now, but don't think we need it for CLM outputs ###
#######################################

#run = Run.from_definition(f'{path_outputs}/{runname}.pfidb')NCLMOUTPUTS = 13 + 4 #13 (number variables) + number of layers over which CLM is active, NZ root

#these 3 entries (year, day start and day end) will eventually be argv to the script so that it can be run from bash script
water_year = 1999
day_start = 364 #day_start = 0 is the first day of the water year, Oct 1 (e.g., day_start = 2 starts at hour 49)
day_end = 365 #day_end = 365 is the final day of the water year, Sept 30

# water_year = int(sys.argv[1])
# day_start = int(sys.argv[1])
# day_end = int(sys.argv[1])

# path to CLM outputs outputs
#path_outputs = '/glade/scratch/tijerina/CONUS2/spinup_WY2003/run_inputs/' #f'/WY{water_year}/'
path_outputs = f'/hydrodata/PFCLM/Taylor/simulations/{water_year}/' #f'/WY{water_year}/'


# PFCLM run name
runname = f'Taylor_{water_year}' #f'CONUS2_{water_year}'

# directory to save averages to
#directory_out = f'/glade/scratch/tijerina/CONUS2/spinup_WY2003/averages'
directory_out = '/home/dtt2/CONUS2/analysis_scripts/Taylor_test_outputs'

# # path to CLM outputs and pfidb
# path_outputs = '/glade/scratch/tijerina/CONUS2/spinup_WY2003/run_inputs/'#f'/WY{water_year}/'
# runname = 'spinup.wy2003' #f'CONUS2_{water_year}'

# # directory to save averages to
# directory_out = f'/glade/scratch/tijerina/CONUS2/spinup_WY2003/averages'
#data = run.data_accessor

# porosity = data.computed_porosity 
# specific_storage = data.specific_storage 
# mannings = data.mannings

# ## remove input filenames for TopoSlopes to force the data accessor to read the output slopes
# ## this fixes a windows issue
# run.TopoSlopesX.FileName = None
# run.TopoSlopesY.FileName = None

# slopex = data.slope_x 
# slopey = data.slope_y 
# mask = data.mask

# # dz_3d = data.dz

# # formatting the mask so that values outside the domain are NA and inside the domain are 1
# # check with mask that has 0 and 1
# active_mask=mask.copy()
# active_mask[active_mask > 0] = 1

In [4]:
####### TAYLOR ########

nz = 5 #10
ny = 47 #3256
nx = 45 #4442

dx = 1000
dy = 1000
dz = 5 # 200
#dz_3d = data.dz

# apparently it's good to use high numbers when saving files to speed up reading?
# for write_pfb function
p = 5 #72
q = 5 #48
r = 1

#### CONUS #######
###READING ALL STATIC VARIABLES NEEDED

# set data accessor time to 1 for reading CLM files
##data.time = 1

# nz = 10
# ny = 3256
# nx = 4442

# dx = 1000
# dy = 1000
# dz = 200
## dz_3d = data.dz

# #apparently it's good to use high numbers when saving files to speed up reading?
# p = 48
# q = 36
# r = 1


#########################

#list of clm variables you want
variables_clm = ['eflx_lh_tot', 'eflx_sh_tot', 'qflx_evap_grnd','qflx_tran_veg', 'swe_out', 't_grnd', 't_soil']
#indication whether you want the mean (1) or the sum (0)
variables_clm_mean = [1,1,0,0,1,1,1] 

ALL_CLM = ['eflx_lh_tot','eflx_lwrad_out','eflx_sh_tot','eflx_soil_grnd','qflx_evap_tot','qflx_evap_grnd','qflx_evap_soi','qflx_evap_veg','qflx_tran_veg','qflx_infl','swe_out','t_grnd','qflx_qirr','t_soil']


### CLM Outputs for reference ###
# eflx_lh_tot    # CLM 1 / Py[0]   # latent heat flux total [W/m2] using the silo variable LatentHeat;
# eflx_lwrad_out # CLM 2 / Py[1]   # outgoing long-wave radiation [W/m2] using the silo variable LongWave;
# eflx_sh_tot    # CLM 3 / Py[2]   # sensible heat flux total [W/m2] using the silo variable SensibleHeat;
# eflx_soil_grnd # CLM 4 / Py[3]   # ground heat flux [W/m2] using the silo variable GroundHeat;
# qflx_evap_tot  # CLM 5 / Py[4]   # total evaporation [mm/s] using the silo variable EvaporationTotal;
# qflx_evap_grnd # CLM 6 / Py[5]   # ground evaporation without condensation [mm/s] using the silo variable EvaporationGround- NoSublimation;
# qflx_evap_soi  # CLM 7 / Py[6]   # soil evaporation [mm/s] using the silo variable EvaporationGround;
# qflx_evap_veg  # CLM 8 / Py[7]   # vegetation evaporation (canopy) and transpiration (mms-1) using the silo variable EvaporationCanopy;
# qflx_tran_veg  # CLM 9 / Py[8]   # vegetation transpiration [mm/s] using the silo variable Transpiration;
# qflx_infl      # CLM 10 / Py[9]  # soil infiltration [mm/s] using the silo variable Infiltration;
# swe_out        # CLM 11 / Py[10] # snow water equivalent [mm] using the silo variable SWE;
# t_grnd         # CLM 12 / Py[11] # ground surface temperature [K] using the silo variable TemperatureGround;
# qflx_qirr      # CLM 13 / Py[12] # irrigation flux
# t_soil         # CLM 


### Fortran reference, can delete later ###
# SWE(:,:) = SWE(:,:) + CLM(:,:,11) / 24.0d0      !avg
# Tgrnd(:,:) = Tgrnd(:,:) + CLM(:,:,12) / 24.0d0      !avg
# ! should I multiply these two fluxes by one hour (3600 s)?
# qflx_grnd(:,:) = qflx_grnd(:,:) + CLM(:,:,6)     !accum
# qflx_trans(:,:) = qflx_trans(:,:) + CLM(:,:,9)   !accum
# Tsoil(:,:) = Tsoil(:,:) + CLM(:,:,14) / 24.0d0      !avg
# LH(:,:) = LH(:,:) + CLM(:,:,1)/ 24.0d0      !avg
# SH(:,:) = SH(:,:) + CLM(:,:,3)/ 24.0d0      !avg



In [5]:
for day in range(day_start,day_end):

    timestamp_day_out = str(int(day+1)).rjust(3, '0')

    ##INITIALIZE WHATEVER DYNAMIC VARIABLES THAT NEED HOURLY READING 
    et = np.zeros((ny,nx)) 
    
    if not variables_clm == False:
        clm_output = np.zeros((NCLMOUTPUTS,ny,nx))
    for h in range(day*24+1,(day+1)*24+1):
        timestamp_reading = str(int(h)).rjust(5, '0')

        #CLM Variables
        clm_output += read_pfb(f'{path_outputs}{runname}.out.clm_output.{timestamp_reading}.C.pfb')
        print(f'reading {path_outputs}{runname}.out.clm_output.{timestamp_reading}.C.pfb')
        
        # Calculate evapotranspiration from qflx_evap_tot
        et += clm_output[4, ...] # NO CONVERSION [mm/s]
        #et += clm_output[4, ...] * 3600 / 1000 #mm/s > mm/h > [m/h]
        #et += clm_output[4, ...] * 3600 / 1000 * dx * dy #mm/s > mm/h > [m^3/h]
        

    # ### compute average for average variables
    et /= 24

    # ### SAVE VARIABLES
    write_pfb(f'{directory_out}/ET.{water_year}.daily.{timestamp_day_out}.pfb',et,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)
   

    #Compute averages CLM outputs
    for ind_clm in range(len(variables_clm)):
        #Check if it's t_soil, then it's 3D!
        if variables_clm[ind_clm] == 't_soil':
            clm_save = clm_output[14:,:,:]
        else:
            ind_in_clmoutput = ALL_CLM.index(variables_clm[ind_clm])
            clm_save = clm_output[ind_in_clmoutput,:,:]
        if variables_clm_mean[ind_clm]==1:
            clm_save/=24
        
        #SAVE VARIABLES CLM outputs
        write_pfb(f'{directory_out}/{variables_clm[ind_clm]}.{water_year}.daily.{timestamp_day_out}.pfb',clm_save,dx=dx,dy=dy,dz=dz,P=p,Q=q,R=r,dist=False)

    

reading /hydrodata/PFCLM/Taylor/simulations/1999/Taylor_1999.out.clm_output.08737.C.pfb
reading /hydrodata/PFCLM/Taylor/simulations/1999/Taylor_1999.out.clm_output.08738.C.pfb
reading /hydrodata/PFCLM/Taylor/simulations/1999/Taylor_1999.out.clm_output.08739.C.pfb
reading /hydrodata/PFCLM/Taylor/simulations/1999/Taylor_1999.out.clm_output.08740.C.pfb
reading /hydrodata/PFCLM/Taylor/simulations/1999/Taylor_1999.out.clm_output.08741.C.pfb
reading /hydrodata/PFCLM/Taylor/simulations/1999/Taylor_1999.out.clm_output.08742.C.pfb
reading /hydrodata/PFCLM/Taylor/simulations/1999/Taylor_1999.out.clm_output.08743.C.pfb
reading /hydrodata/PFCLM/Taylor/simulations/1999/Taylor_1999.out.clm_output.08744.C.pfb
reading /hydrodata/PFCLM/Taylor/simulations/1999/Taylor_1999.out.clm_output.08745.C.pfb
reading /hydrodata/PFCLM/Taylor/simulations/1999/Taylor_1999.out.clm_output.08746.C.pfb
reading /hydrodata/PFCLM/Taylor/simulations/1999/Taylor_1999.out.clm_output.08747.C.pfb
reading /hydrodata/PFCLM/Taylor/

In [6]:
clm_output[4,20:22,20:22]

array([[4.47354074e-04, 6.57464670e-05],
       [4.47279567e-04, 4.50862310e-04]])

In [7]:
day1_ET = read_pfb(f'{directory_out}/ET.1999.daily.001.pfb')

In [8]:
day1_ET.shape

(1, 47, 45)

In [9]:
day1_ET[0,20:25,20:25]

array([[0.00222829, 0.00191517, 0.00191714, 0.00211404, 0.0021388 ],
       [0.00221075, 0.0022116 , 0.00221264, 0.00215943, 0.00222501],
       [0.00192433, 0.00213168, 0.00220674, 0.00220976, 0.00221751],
       [0.00191311, 0.00189339, 0.00220083, 0.00220568, 0.00212568],
       [0.00186233, 0.00186869, 0.00219993, 0.00220119, 0.002209  ]])

In [10]:
soil_temp = read_pfb(f'{directory_out}/t_soil.1999.daily.001.pfb')

In [11]:
soil_temp.shape

(3, 47, 45)

In [12]:
#SWE
swe = np.squeeze(read_pfb(f'{directory_out}/swe_out.1999.daily.001.pfb'))
swe.shape

(47, 45)

In [13]:
swe[20:25,20:25]

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])